In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
from mhkit import wave
import pandas as pd
import os
import yaml
import re
from pathlib import Path
from scipy.special import hankel1 as Hankel

ArgumentError: ArgumentError: Package matplotlib not found in current path.
- Run `import Pkg; Pkg.add("matplotlib")` to install the matplotlib package.

In [ ]:
# Determine root_dir
if "__file__" in globals():
    # Running as a script
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()

# Traverse up to WEC-DECIDER root
root_dir = os.path.abspath(os.path.join(script_dir, "..", ".."))  
cem_dir = os.path.join(root_dir, "modules", "CEM")

In [ ]:

data_type = '3-hour'
year = [2010]
parameters = [
    'omni-directional_wave_power',
    'significant_wave_height',
    'energy_period',
    'directionality_coefficient',
    'maximum_energy_direction',
    'mean_absolute_period',
    'mean_zero-crossing_period',
    'peak_period',
    'spectral_width'
]
lat_lon = (43.5, -70) # off coast of Maine
data, metadata = wave.io.hindcast.hindcast.request_wpto_point_data(data_type, parameters, lat_lon, year)
data.head()

In [ ]:
data_mod = data.copy()

rho_w = 1025  # kg/m^3, density of water
g = 9.81  # m/s^2, acceleration due to gravity
#J_calc = data_mod["significant_wave_height_0"]**2 * rho_w * g**2 / (64 * np.pi) * data_mod["energy_period_0"]
J_calc = data_mod["significant_wave_height_0"]**2 * rho_w * g**2 / (64 * np.pi) * data_mod["mean_zero-crossing_period_0"]
fudge = 1 - .007 * (data_mod["mean_zero-crossing_period_0"] - (2*np.pi)) 
data_mod["Power_density_predicted"] = J_calc / fudge #/ data_mod["spectral_width_0"] #* data_mod["directionality_coefficient_0"]
data_mod["ratio_power_density"] = data_mod["omni-directional_wave_power_0"] / data_mod["Power_density_predicted"]

data_mod.head()


In [ ]:
fig, ax = plt.subplots(1, parameters.__len__(), sharey=True, figsize=(25, 4))
units = ["W/m^2", "m", "s", "-", "deg", "s", "s", "s", "-"]
for i in range(parameters.__len__()):
    col = parameters[i] + "_0"
    if False: #"period" in col and col != "energy_period_0":
        normalize = True
        norm = data_mod["energy_period_0"]
    else:
        norm = 1
        normalize = False
    x_data = data_mod[col] / norm
    ax[i].scatter(x_data, 1/data_mod["ratio_power_density"])
    label = parameters[i] + " (" + units[i] + ")"
    if normalize:
        label += " / " + "energy_period" + " (s)"
    ax[i].set_xlabel(label)
    ax[i].plot([x_data.min(), x_data.max()], [1, 1], color='k')
ax[0].set_ylabel("J calc / J ")

In [ ]:
# capture width
G = 1 # gain for heave (=2 for surge/pitch)
omega = 2 * np.pi / data_mod["mean_zero-crossing_period_0"]

zeta_range = [0.05, 0.15, 0.25]#[0.01, 0.05, 0.1, 0.2, 1]
omega_n_range = [0.5, 0.6, 0.7]#[0.4, 0.5, 0.6, 0.7, 1.5]
float_diam_range = [20] #[5, 15, 25, 35]  # in meters

ZETA, OMEGA, OMEGA_N, D_f = np.meshgrid(zeta_range, omega, omega_n_range, float_diam_range)

h = 85 # water depth in meters
k = wave.resource.wave_number(omega/ (2*np.pi), h)  # wave number
_,K,_,_ = np.meshgrid(zeta_range, k, omega_n_range, float_diam_range) # wave number mesh, to mimic omega mesh
mag_B_0_e_nondim = -0.0898 * K + 0.0217 # see fit in MDOcean/dev/B_0_e_vs_frequency.m
mag_B_0_e = mag_B_0_e_nondim * h 

denom = np.cosh(K*h) * np.abs(Hankel(0,K*D_f/2))
N_0 = 1 / 2 * (1 + np.sinh(2 * K * h) / (2 * K * h))
mag_F_over_eta = 4 * rho_w * g * h * N_0**(1/2) * mag_B_0_e / denom
curly_D = np.tanh(K*h) + K*h*(1 - np.tanh(K*h)**2)
B_h = mag_F_over_eta**2 / (2 * rho_w * g**2) * K * OMEGA / curly_D
B_u = B_h
CW_max = G * g / OMEGA**2 # radiation limit

real = 1 - (OMEGA / OMEGA_N)**2
imag = 2 * ZETA * OMEGA / OMEGA_N
mag_RAO_sq = 1/( real**2 + imag**2 )
CW_frac_raw = 2 * B_u * OMEGA**5 * mag_RAO_sq / (G * g**3 * rho_w)
CW_frac = np.minimum(CW_frac_raw, 1)
CW = CW_frac * CW_max

In [ ]:
# bode plot to check mag RAO

for d_idx in range(len(float_diam_range)):
    fig, ax = plt.subplots()
    for z_idx in range(len(zeta_range)):
        for o_idx in range(len(omega_n_range)):
        
            x_data = OMEGA[:, z_idx, o_idx, d_idx]
            y_data = 20 * np.log10(np.sqrt(mag_RAO_sq[:, z_idx, o_idx, d_idx]))
            sorted_indices = np.argsort(x_data)
            x_data = x_data[sorted_indices]
            y_data = y_data[sorted_indices]
            ax.semilogx(x_data, y_data, label="$\zeta =$ " + str(zeta_range[z_idx]) + ", $\omega_n =$ " + str(omega_n_range[o_idx]))
    ax.set_xlabel("Frequency (rad/s)")
    ax.set_ylabel("Magnitude (dB)")
    ax.set_title("Bode plot for float diameter = " + str(float_diam_range[d_idx]) + " m")
    ax.legend()

In [ ]:
# plot time series of CW fraction
for d_idx in range(len(float_diam_range)):
    fig, ax = plt.subplots(len(zeta_range), len(omega_n_range), sharey=True, figsize=(10,8))
    for z_idx in range(len(zeta_range)):
        for o_idx in range(len(omega_n_range)):
            # deal with axes size for scalars
            if len(zeta_range) == 1 and len(omega_n_range) == 1:
                ax_obj = ax
            elif len(zeta_range) == 1:
                ax_obj = ax[o_idx]
            elif len(omega_n_range) == 1:
                ax_obj = ax[z_idx]
            else:
                ax_obj = ax[z_idx, o_idx]

            ax_obj.plot(CW_frac_raw[:, z_idx, o_idx])
            ax_obj.plot([0, len(CW_frac_raw[:, z_idx, o_idx])], [1, 1], 'k')
            if z_idx == 0:
                ax_obj.set_title("$\omega_n =$ " + str(omega_n_range[o_idx]))
            if o_idx == 0:
                ax_obj.set_ylabel("$\zeta =$ " + str(zeta_range[z_idx]))
            ax_obj.set_ylim([0, 1.5])
    fig.supxlabel("time index (3-hour intervals)")
    fig.supylabel("Raw CW fraction for $D_f =$ " + str(float_diam_range[d_idx]) + " m")

In [ ]:
# Apply different power limits for each zeta and omega_n
P_lim_list = np.concatenate( (np.linspace(0.005, 0.025, num=5), np.linspace(0.03, 0.09, num=7), np.linspace(0.1,0.9,num=9)) )

for d_idx in range(len(float_diam_range)):
    for z_idx in range(len(zeta_range)):
        for o_idx in range(len(omega_n_range)):
            key_unlimited = "Avail_zeta_" + str(zeta_range[z_idx]) + \
                            "_omega_n_"   + str(omega_n_range[o_idx]) + \
                            "_D_f_"       + str(float_diam_range[d_idx])
            data_unlimited = CW_frac[:, z_idx, o_idx, d_idx] / np.max(CW_frac[:, z_idx, o_idx, d_idx])
            for P_limit in P_lim_list:
                key_limited = key_unlimited + "_limited_" + str(int(P_limit*1000))
                data_mod[key_limited] = np.minimum(data_unlimited, P_limit) / P_limit
            data_mod[key_unlimited] = data_unlimited

data_mod = data_mod.copy() # unfragment dataframe
pd.set_option('display.max_rows', None)
means = data_mod.mean()
num_cols_added = 2
CFs = np.array(means)[(len(parameters)+num_cols_added):]
print(means)
print("Capacity factors: ",CFs)

In [ ]:

P_lim_list_full = np.concatenate((P_lim_list,[1]))
lens = (len(P_lim_list_full),len(zeta_range), len(omega_n_range))
P_lim_frac = np.broadcast_to(P_lim_list_full[:,None,None], lens)
CF_matrix = np.reshape(CFs, (lens[1],lens[2],lens[0])).transpose(2, 0, 1)

cols = ['r','b','g']
lines = ['-', '--', ':']
fig, ax = plt.subplots()
for z_idx in range(len(zeta_range)):
    for o_idx in range(len(omega_n_range)):
        ax.plot(P_lim_frac[:, z_idx, o_idx], CF_matrix[:, z_idx, o_idx], color=cols[z_idx], linestyle=lines[o_idx],
                label="$\zeta =$ " + str(zeta_range[z_idx]) + ", $\omega_n =$ " + str(omega_n_range[o_idx]))
ax.set_ylabel('Capacity Factor')
ax.set_xlabel('Power Limit Fraction')

ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
ax.grid()
plt.show()

In [ ]:
"""
fig, ax = plt.subplots()
plt.hist(data_mod.Avail, bins=200, density=True)
plt.xlabel('Normalized Power')
plt.ylabel('Probability Density')

fig, ax = plt.subplots()
plt.hist(data_mod.Avail, bins=200, density=True)
plt.plot([0.02, 0.02],[0,12],'k--', [0.04,0.04],[0,4.5],'k--', [0.06,0.06],[0,2.2],'k--',
          [0.08,0.08],[0,1.5],'k--', [0.1,0.1],[0,1.5],'k--')
plt.xlim([0,0.12])
"""

In [ ]:
cap_costs = 1e3 * np.array([400, 500, 600])# 2000, 8500, 15000]) # USD/MW. Wind is around 1e3 * 1,600, nominal RM3 is around 1e3 * 13,000
years = [2030,2045]
P_lims = np.array([0.1, 0.4, 0.7, 1.0])

cap_cost_mesh, year_mesh = np.meshgrid(cap_costs, years)
cap_cost_vec = cap_cost_mesh.flatten()
year_vec = year_mesh.flatten()

print(cap_cost_vec)
print(year_vec)

wave_cost_str = [f"Wave_{cap_cost/1e3:.0f}" for cap_cost in cap_costs] # $k/MW
print(wave_cost_str)
wave_cost_str_repeated = np.matlib.repmat(wave_cost_str, len(years), 1).flatten()

print(wave_cost_str_repeated)

#P_lim_string = (P_lim_vec*1000).astype(int).astype(str)
#keys = np.char.add( np.repeat("Avail_limited_",np.size(P_lim_vec)), P_lim_string )
#print(keys)

# use CF to lookup power limit and corresponding availability timeseries

In [ ]:
# make a separate csv per profile
variability_names = []
location_folder = cem_dir + '/data_east/'
profile_folder = location_folder + 'wave_profiles/'
if not os.path.exists(profile_folder):
    os.makedirs(profile_folder)

csv_num = 0

for d_idx in range(len(float_diam_range)):
    for P_limit in P_lims:
        for z_idx in range(len(zeta_range)):
            for o_idx in range(len(omega_n_range)):
                key_unlimited = "Avail_zeta_" + str(zeta_range[z_idx]) + "_omega_n_" + str(omega_n_range[o_idx]) + "_D_f_" + str(float_diam_range[d_idx])
                key_limited = key_unlimited + "_limited_" + str(int(P_limit*1000))
                if P_limit == 1:
                    key = key_unlimited
                else:
                    key = key_limited
                profile_data = data_mod[key].values
                profile_hourly = np.interp(np.arange(0, 8760), np.arange(0, 8760, 3), profile_data)

                csv_data = pd.DataFrame()
                csv_data["technology"] = wave_cost_str_repeated
                csv_data["planning_year"] = year_vec
                csv_data["capex_mw"] = cap_cost_vec
                csv_data["capex_mwh"] = 0
                csv_data["fixed_o_m_mw"] = cap_cost_vec / 10
                csv_data["fixed_o_m_mwh"] = 0
                csv_data["variable_o_m_mwh"] = 0
                csv_data["wacc_real"] = 0.07
                csv_data["dollar_year"] = 2014
                csv_data["heat_rate"] = 0
                csv_data["profile"] = np.array2string(profile_hourly, separator=',', precision=4, threshold=np.inf)

                csv_data.to_csv(profile_folder + key_limited +'.csv',index=False)
                variability_names.append(key_limited)
                csv_num += 1
                print(f"Wrote profile csv {csv_num} of {len(float_diam_range) * len(P_lims) * len(zeta_range) * len(omega_n_range)}")

csv_data.head()


In [ ]:
# make a scenario inputs csv

# grid scenarios (2 dof, 3 options each): electrification, carbon_constraint
# year (1 dof, 2 options)
# design scenarios (2 dof capturing 4 dof, with 3, 4, 5, and 3 options): wave variability (zeta, omega_n, and power limit), wave cost

# total 9 grid scenarios, 2 years, 180 design scenarios = 3240 scenario rows

electrification = ['ref','med','high']
carbon_constraint = ['low','med','high']
scenario_vals = (electrification, carbon_constraint, variability_names, wave_cost_str) # all possible values, for yaml

# values to use in this sweep
indices_in_use = [[0],[1],range(len(variability_names)),range(len(wave_cost_str))]
#indices_in_use = [[0],[1],[0],range(len(wave_cost_str))] # single variability
idx_years_used = 0
scenario_vals_used = tuple(
    [[scenario_vals[i][idx] for idx in idx_list] for i, idx_list in enumerate(indices_in_use)]
)
scenario_mesh_tuple = np.meshgrid(years[idx_years_used], *scenario_vals_used, indexing='ij')

scenario_col_names = ['year', 'electrification', 'carbon_constraint', 'wave_variability', 'wave_cost']

scenario_inputs = pd.DataFrame()
num_scenarios = len(scenario_mesh_tuple[0].flatten())



scenario_inputs[scenario_col_names] = np.array([scenario_col.flatten() for scenario_col in scenario_mesh_tuple]).T

#scenario_inputs["case_id"] = ['Case_' + str(i) for i in np.arange(1, num_scenarios + 1)]
scenario_inputs["case_id"] = ['Case_' + str('_'.join(str(k) + '_' + str(v) for k, v in row.items())) for row in scenario_inputs.to_dict('records')]

print(len(scenario_inputs))
scenario_inputs.head()
scenario_inputs.to_csv(location_folder + 'scenario_input.csv',index=False)


In [ ]:
# make a settings management for power genome

pg_settings_names = ['electrification_scenario', 'emission_policies_fn', 'additional_technologies_fn', 'additional_new_gen']

elec_val_names = ['REFERENCE ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT', 'MEDIUM ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT', 'HIGH ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT']
carbon_val_names = ['emission_policies_net_zero.csv','emission_policies.csv','emission_policies_no_limit.csv']
wave_var_names = ['wave_profiles/' + key + '.csv' for key in variability_names]
wave_cost_val_names = [[str] for str in wave_cost_str] # turn list of strs into list of lists

pg_val_names = [elec_val_names, carbon_val_names, wave_var_names, wave_cost_val_names]

idxs = range(len(scenario_col_names)-1)
scenario_inner_dicts = [dict(zip(scenario_vals[i], \
                               [{pg_settings_names[i]:val} for val in pg_val_names[i]] \
                             )) for i in idxs]
scenario_dict = dict(zip(scenario_col_names[1:], scenario_inner_dicts))
settings_management = {'settings_management': {year: scenario_dict for year in years}}

print(f"scenario inner dicts: {scenario_inner_dicts}")
print(f"scenario dict: {scenario_dict}")
print(f"settings management: {settings_management}")

with open(location_folder + 'settings/settings_management.yml', 'w') as f:
    yaml.dump(settings_management, f, sort_keys=False)



In [ ]:
# make a cost multiplier yaml for power genome
def update_cost_mult_yaml(yaml_file_path, new_yaml_path, anchor_name="offshore_wind"):
    """
    Update the YAML file with new offshore wind technologies.
    
    Args:
        yaml_file_path (str or Path): Path to the YAML file
        new_yaml_path (str or Path): Path to save the updated YAML file
        anchor_name (str): Name of the YAML anchor to update
    """
    yaml_file_path = Path(yaml_file_path)
    new_yaml_path = Path(new_yaml_path)
    
    # Generate the new offshore wind list
    offshore_wind_list = ["OffShoreWind_Class3"] + wave_cost_str
    print(f"Generated offshore wind list: {offshore_wind_list}")
    
    # Read the current file
    with open(yaml_file_path, 'r') as f:
        content = f.read()
    
    print(f"Original file size: {len(content)} characters")
    print(f"Looking for anchor: {anchor_name}")
    
    # Check if the anchor exists in the file
    anchor_pattern = f'{anchor_name}: &{anchor_name}'
    if anchor_pattern not in content:
        print(f"Warning: Anchor '{anchor_pattern}' not found in file!")
        print("Available content preview:")
        print(content[:500] + "..." if len(content) > 500 else content)
        return
    
    # Improved pattern to match the offshore wind anchor section
    # This matches from the anchor definition to the end of the list
    pattern = rf'({anchor_name}: &{anchor_name}\n)(.*?)(?=\n\S|\n*$)' 
    
    # Create the new YAML section
    new_section = f"{anchor_name}: &{anchor_name}\n"
    for tech in offshore_wind_list:
        new_section += f"  - {tech}\n"
    
    print(f"New section to insert:\n{new_section}")
    
    # Replace the section
    new_content = re.sub(pattern, new_section.rstrip() + '\n', content, flags=re.DOTALL)
    
    # Write back to file
    with open(new_yaml_path, 'w') as f:
        f.write(new_content)

update_cost_mult_yaml(cem_dir + '/template/cost_mult_tech_map.yml', location_folder + '/settings/cost_mult_tech_map_updated.yml')

using Pkg
Pkg.activate("GenX")
]
include("caserunner.jl")